# Module Five Assignment: Cartpole Problem
Review the code in this notebook and in the score_logger.py file in the *scores* folder (directory). Once you have reviewed the code, return to this notebook and select **Cell** and then **Run All** from the menu bar to run this code. The code takes several minutes to run.

Note: If the code is running properly, you should begin to see output appearing above this code block. It will take several minutes, so it is recommended that you let this code run in the background while completing other work. When the code has finished, it will print output saying, "Solved in _ runs, _ total runs."

You may see an error about not having an exit command. This error does not affect the program's functionality and results from the steps taken to convert the code from Python 2.x to Python 3. Please disregard this error.



In [27]:
import random  
import gym  
import numpy as np  
from collections import deque  
from keras.models import Sequential  
from keras.layers import Dense  
from keras.optimizers import Adam  
  
from scores.score_logger import ScoreLogger 

# modifying learning rate from .001 to .01
ENV_NAME = "CartPole-v1"  
  
GAMMA = 0.95  
LEARNING_RATE = 0.004 # increased from .001
  
MEMORY_SIZE = 1000000  
BATCH_SIZE = 20  
  
EXPLORATION_MAX = 1.0  
EXPLORATION_MIN = 0.01  
EXPLORATION_DECAY = 0.995  
  
  
class DQNSolver:  
  
    def __init__(self, observation_space, action_space):  
        self.exploration_rate = EXPLORATION_MAX  
  
        self.action_space = action_space  
        self.memory = deque(maxlen=MEMORY_SIZE)  
  
        self.model = Sequential()  
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))  
        self.model.add(Dense(24, activation="relu"))  
        self.model.add(Dense(self.action_space, activation="linear"))  
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))  
  
    def remember(self, state, action, reward, next_state, done):  
        self.memory.append((state, action, reward, next_state, done))  
  
    def act(self, state):  
        if np.random.rand() < self.exploration_rate:  
            return random.randrange(self.action_space)  
        q_values = self.model.predict(state)  
        return np.argmax(q_values[0])  
  
    def experience_replay(self):  
        if len(self.memory) < BATCH_SIZE:  
            return  
        batch = random.sample(self.memory, BATCH_SIZE)  
        for state, action, reward, state_next, terminal in batch:  
            q_update = reward  
            if not terminal:  
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))  
            q_values = self.model.predict(state)  
            q_values[0][action] = q_update  
            self.model.fit(state, q_values, verbose=0)  
        self.exploration_rate *= EXPLORATION_DECAY  
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)  
  
  
def cartpole():  
    env = gym.make(ENV_NAME)  
    score_logger = ScoreLogger(ENV_NAME)  
    observation_space = env.observation_space.shape[0]  
    action_space = env.action_space.n  
    dqn_solver = DQNSolver(observation_space, action_space)  
    run = 0  
    while True:  
        run += 1  
        state = env.reset()  
        state = np.reshape(state, [1, observation_space])  
        step = 0  
        while True:  
            step += 1  
            #env.render()  
            action = dqn_solver.act(state)  
            state_next, reward, terminal, info = env.step(action)  
            reward = reward if not terminal else -reward  
            state_next = np.reshape(state_next, [1, observation_space])  
            dqn_solver.remember(state, action, reward, state_next, terminal)  
            state = state_next  
            if terminal:  
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))  
                score_logger.add_score(step, run)  
                break  
            dqn_solver.experience_replay()


In [28]:
 cartpole()

Run: 1, exploration: 1.0, score: 13
Scores: (min: 13, avg: 13, max: 13)

Run: 2, exploration: 0.9091562615825302, score: 26
Scores: (min: 13, avg: 19.5, max: 26)

Run: 3, exploration: 0.7590483508202912, score: 37
Scores: (min: 13, avg: 25.333333333333332, max: 37)

Run: 4, exploration: 0.7255664080186093, score: 10
Scores: (min: 10, avg: 21.5, max: 37)

Run: 5, exploration: 0.6662995813682115, score: 18
Scores: (min: 10, avg: 20.8, max: 37)

Run: 6, exploration: 0.6027415843082742, score: 21
Scores: (min: 10, avg: 20.833333333333332, max: 37)

Run: 7, exploration: 0.5790496471185967, score: 9
Scores: (min: 9, avg: 19.142857142857142, max: 37)

Run: 8, exploration: 0.547986285490042, score: 12
Scores: (min: 9, avg: 18.25, max: 37)

Run: 9, exploration: 0.5238143793828016, score: 10
Scores: (min: 9, avg: 17.333333333333332, max: 37)

Run: 10, exploration: 0.4883155414435353, score: 15
Scores: (min: 9, avg: 17.1, max: 37)

Run: 11, exploration: 0.4506816115185697, score: 17
Scores: (min:

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.100986). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Run: 276, exploration: 0.01, score: 87
Scores: (min: 8, avg: 179.05, max: 500)

Run: 277, exploration: 0.01, score: 164
Scores: (min: 8, avg: 177.3, max: 500)

Run: 278, exploration: 0.01, score: 17
Scores: (min: 8, avg: 177.32, max: 500)

Run: 279, exploration: 0.01, score: 24
Scores: (min: 8, avg: 177.47, max: 500)

Run: 280, exploration: 0.01, score: 13
Scores: (min: 8, avg: 177.48, max: 500)

Run: 281, exploration: 0.01, score: 10
Scores: (min: 8, avg: 176.9, max: 500)

Run: 282, exploration: 0.01, score: 27
Scores: (min: 8, avg: 173.44, max: 500)

Run: 283, exploration: 0.01, score: 180
Scores: (min: 8, avg: 174.39, max: 500)

Run: 284, exploration: 0.01, score: 172
Scores: (min: 8, avg: 175.22, max: 500)

Run: 285, exploration: 0.01, score: 47
Scores: (min: 8, avg: 175.59, max: 500)

Run: 286, exploration: 0.01, score: 20
Scores: (min: 8, avg: 175.7, max: 500)

Run: 287, exploration: 0.01, score: 203
Scores: (min: 8, avg: 172.82, max: 500)

Run: 288, exploration: 0.01, score: 14


NameError: name 'exit' is not defined